## Get data from an open source:
- https://www.goodreads.com/quotes
- access all the asked tags

In [53]:
import requests
res = requests.get('https://www.goodreads.com/quotes')

In [54]:
soup = bs4.BeautifulSoup(res.text, 'html.parser') #there are many types of parser

Get all the author names:
- Approach1: using select

In [59]:
soup.select(".authorOrTitle")[:3]
for name in soup.select(".authorOrTitle")[:3]:
    print(name.text)


    Marilyn Monroe
  

    Oscar Wilde
  

    Albert Einstein
  


Approach2: OR, by using find_all we can do it:

In [80]:
soup.find_all('span', class_='authorOrTitle')[:3]
for name in soup.find_all('span', class_='authorOrTitle')[:3]:
    print(name.text)


    Marilyn Monroe
  

    Oscar Wilde
  

    Albert Einstein
  


In [81]:
#len(soup.find_all('span', class_='authorOrTitle'))

30

OR:

In [60]:
Author = []
soup.find_all('span', class_='authorOrTitle')
for name in soup.find_all('span', class_='authorOrTitle'):
    Author.append(name.text.strip())
Author[:3]

['Marilyn Monroe', 'Oscar Wilde', 'Albert Einstein']

In [ ]:
#Get the likes
tag = a
class = smallText

In [94]:
likes = []
soup.find_all('a', class_='smallText')
for name in soup.find_all('a', class_='smallText'):
    likes.append(name.text.strip())
max(likes)

'97778 likes'

In [61]:
quotes = []
soup.find_all('div', class_='quoteText')
for name in soup.find_all('div', class_='quoteText'):
    quotes.append(name.text.strip())
len(quotes)

30

In [62]:
quotes[:2]

["“I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”\n    ―\n  \n    Marilyn Monroe",
 '“Be yourself; everyone else is already taken.”\n    ―\n  \n    Oscar Wilde']

- So, it seems quotes are appearing along with the writer name as well.
- But we want only the quote part:
- `As under the same tag 'div' and same class 'quoteText' there are sub tags also, hence we need to use 'children' here inorder to get the quotes part only.`

In [68]:
print(soup.find_all('div', class_='quoteText')[0])  #print the 1st quote
print('******************************************************************')
list(soup.find_all('div', class_='quoteText')[0].children)

<div class="quoteText">
      “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”
  <br/>  ―
  <span class="authorOrTitle">
    Marilyn Monroe
  </span>
</div>
******************************************************************


["\n      “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”\n  ",
 <br/>,
 '  ―\n  ',
 <span class="authorOrTitle">
     Marilyn Monroe
   </span>,
 '\n']

- So, it seems that it has many child tags as well.
- We need only the 1st: Need to select the 0th children

In [66]:
#how many childern are there under this div:
len(list(soup.find_all('div', class_='quoteText')[0].children))   #selected 1st quote and it's total chidlren tags

5

In [101]:
#Now select the 1st one only out of the total 5 children:
list(soup.find_all('div', class_='quoteText')[0].children)[0]

"\n      “I'm selfish, impatient and a little insecure. I make mistakes, I am out of control and at times hard to handle. But if you can't handle me at my worst, then you sure as hell don't deserve me at my best.”\n  "

In [109]:
#So we can write this way to get all the children:
quotes2 = []
for name in soup.find_all('div', class_='quoteText'):
    quotes2.append(list(name.children)[0].strip())
len(quotes2)

30

In [112]:
likes = []
soup.find_all('a', class_='smallText')
for name in soup.find_all('a', class_='smallText'):
    likes.append(list(name.children)[0].strip())
max(likes)

'97778 likes'

In [116]:
#now want only the number part:
soup.find_all('a', class_='smallText')[0].text.split()[0]

'153875'

In [120]:
likes2 = []
for i in soup.find_all('a', class_='smallText'):
    likes2.append((i.text.split()[0]))

In [119]:
import pandas as pd

Put 3 columns into a df: manual way

In [122]:
df = pd.DataFrame({'Author':Author,'quotes':quotes2,'likes':likes2})
df.head()

,Author,quotes,likes
0,Marilyn Monroe,"“I'm selfish, impatient and a little insecure....",153875
1,Oscar Wilde,“Be yourself; everyone else is already taken.”,153849
2,Albert Einstein,“Two things are infinite: the universe and hum...,135485
3,Frank Zappa,"“So many books, so little time.”",132111
4,Marcus Tullius Cicero,“A room without books is like a body without a...,123427


- Now if we want to do this for each page of this website then we need some automatic way rather than doing it manually.
- Put 3 columns into a df: automatic way (use customized func instead)

In [149]:
def get_table(soup):
    Author = []
    soup.find_all('span', class_='authorOrTitle')
    for name in soup.find_all('span', class_='authorOrTitle'):
        Author.append(name.text.strip())
        
    quotes2 = []
    for name in soup.find_all('div', class_='quoteText'):
        quotes2.append(list(name.children)[0].strip())
        
    likes2 = []
    for i in soup.find_all('a', class_='smallText'):
        likes2.append((i.text.split()[0]))
        
    import pandas as pd
    df = pd.DataFrame({'Author':Author,'quotes':quotes2,'likes':likes2})
    return (df)

In [126]:
get_table(soup)

,Author,quotes,likes
0,Marilyn Monroe,"“I'm selfish, impatient and a little insecure....",153875
1,Oscar Wilde,“Be yourself; everyone else is already taken.”,153849
2,Albert Einstein,“Two things are infinite: the universe and hum...,135485
3,Frank Zappa,"“So many books, so little time.”",132111
4,Marcus Tullius Cicero,“A room without books is like a body without a...,123427


**Now for multiple page scrapping how to get the same resluts:**

In [127]:
stub = "https://www.goodreads.com/quotes?page={}"

In [130]:
print(stub.format(1))
#hence getting page1

print(stub.format(4))
#hence getting page4

https://www.goodreads.com/quotes?page=1
https://www.goodreads.com/quotes?page=4


In [133]:
#to access each and every page initiate a loop: access only 1 to 10 pages
for page in range(1,11):
    url = stub.format(page)
    print(url)

https://www.goodreads.com/quotes?page=1
https://www.goodreads.com/quotes?page=2
https://www.goodreads.com/quotes?page=3
https://www.goodreads.com/quotes?page=4
https://www.goodreads.com/quotes?page=5
https://www.goodreads.com/quotes?page=6
https://www.goodreads.com/quotes?page=7
https://www.goodreads.com/quotes?page=8
https://www.goodreads.com/quotes?page=9
https://www.goodreads.com/quotes?page=10


In [150]:
#Now request:
table = []
for page in range(1,11):
    url = stub.format(page)
    html = requests.get(url).text
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table.append(get_table(soup))

In [151]:
len(table)
#extracted from 10 diff pages

10

In [152]:
#combine those:
data = pd.concat(table, axis=0)

In [153]:
data.shape

(300, 3)

Keep track how much data is being processed:
- use lib tqdm

In [154]:
#extract for 20 pages:
from tqdm import tqdm

In [155]:
#extract from 20 pages keeping track on it:
table = []
for page in tqdm(range(1,21)):
    url = stub.format(page)
    html = requests.get(url).text
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table.append(get_table(soup))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:41<00:00,  2.07s/it]


In [159]:
#combine those and check the shape now:
data = pd.concat(table, axis=0)
data.shape

(600, 3)

Another websit:

In [223]:
stub = 'https://www.goodreads.com/list/show/19.Best_for_Book_Club'

In [224]:
html = requests.get(stub).text
soup = bs4.BeautifulSoup(html, 'html.parser')
#table.append(get_table(soup))

get : book names, aithor anme, ratings
- book tag - a, class- bookTitle
- author tag - a, class- authorName
- ratinsg tag- span, class- minirating

In [174]:
len(soup.find_all('a', class_='bookTitle'))

100

In [225]:
books = []
for i in soup.find_all('a', class_='bookTitle'):
    books.append((i.text.strip()))
len(books)

100

In [226]:
authors = []
for i in soup.find_all('a', class_='authorName'):
    authors.append((i.text.strip()))
len(authors)

100

In [227]:
ratings = []
for i in soup.find_all('span', class_='minirating'):
    ratings.append((i.text.strip().split()[0]))
len(ratings)

100

In [195]:
#OR
#get only the ratings:
ratings_no = []
for i in soup.find_all('span', class_='minirating'):
    ratings_no.append((i.text.split()[0]))
ratings_no[:2]

Need to customize the func accordingly:

In [206]:
def get_table(soup):
    books = []
    for i in soup.find_all('a', class_='bookTitle'):
        books.append((i.text.strip()))
        
    authors = []
    for i in soup.find_all('a', class_='authorName'):
        authors.append((i.text.strip()))
        
    ratings_no = []
    for i in soup.find_all('span', class_='minirating'):
        ratings_no.append((i.text.split()[0]))
        
    import pandas as pd
    df1 = pd.DataFrame({'Books':books,'Authors':authors,'Ratings':ratings})
    return (df1)

In [217]:
df2 = get_table(soup)
df2.shape

(100, 3)

In [218]:
df2.head()

,Books,Authors,Ratings
0,The Help,Kathryn Stockett,4.46
1,The Kite Runner,Khaled Hosseini,4.31
2,Water for Elephants,Sara Gruen,4.09
3,The Book Thief,Markus Zusak,4.38
4,To Kill a Mockingbird,Harper Lee,4.28


In [221]:
df2['Ratings'] = df2['Ratings'].astype(float)

In [222]:
df2['Ratings'].max()

4.62

So far we got from 1 web page only. Now get from other pages as well.

In [379]:
#step1: create the stub at first:
stub = "https://www.goodreads.com/list/show/19.Best_for_Book_Club={}"

In [380]:
stub.format(5)

'https://www.goodreads.com/list/show/19.Best_for_Book_Club=5'

In [230]:
#Step2: extract from 20 pages keeping track on it:
table = []
for page in tqdm(range(1,21)):
    url = stub.format(page)
    html = requests.get(url).text
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table.append(get_table(soup))

100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [01:35<00:00,  4.78s/it]


In [231]:
#Step3: combine those and check the shape now:
data = pd.concat(table, axis=0)
data.shape

(2000, 3)

In [232]:
data.head()

,Books,Authors,Ratings
0,The Help,Kathryn Stockett,4.46
1,The Kite Runner,Khaled Hosseini,4.31
2,Water for Elephants,Sara Gruen,4.09
3,The Book Thief,Markus Zusak,4.38
4,To Kill a Mockingbird,Harper Lee,4.28


another: http://books.toscrape.com/

In [ ]:
Task:
get all the book name, price an dratings
book tag -a , class title
price tag- div, class product_price
rating tag- , class

In [69]:
url = "http://books.toscrape.com/"
html = requests.get(url).text
soup = bs4.BeautifulSoup(html, 'html.parser')

In [328]:
#soup.find_all('div')[0]

In [329]:
price = []
for i in soup.find_all('div', class_='product_price'):
    price.append((i.text.strip().split()[0].replace('Â£','')))
print(len(price))
print(price[:3])

20
['51.77', '53.74', '50.10']


In [73]:
for i in soup.find_all('article','product_pod')[:5]:
    print(i.p.get('class'))

['star-rating', 'Three']
['star-rating', 'One']
['star-rating', 'One']
['star-rating', 'Four']
['star-rating', 'Five']


In [330]:
rating = []
for i in soup.find_all('article','product_pod'):
    rating.append(i.p.get('class')[1])
print(len(rating))
print(rating[:3])

20
['Three', 'One', 'One']


In [331]:
books2 = []
for i in soup.find_all('h3'):
    books2.append((i.text.strip()))
print(len(books2))
for i in books2[:3]:
    print(i)

20
A Light in the ...
Tipping the Velvet
Soumission


In [324]:
def get_table(soup):
    books2 = []
    for i in soup.find_all('h3'):
        books2.append((i.text.strip()))
        
    price = []
    for i in soup.find_all('div', class_='product_price'):
        price.append((i.text.strip().split()[0].replace('Â£','')))
        
    rating = []
    for i in soup.find_all('article','product_pod'):
        rating.append(i.p.get('class')[1])
        
    import pandas as pd
    df1 = pd.DataFrame({'Books':books2,'Price':price,'Rating':rating})
    return (df1)

In [326]:
get_table(soup)[:5]

,Books,Price,Rating
0,A Light in the ...,51.77,Three
1,Tipping the Velvet,53.74,One
2,Soumission,50.10,One
3,Sharp Objects,47.82,Four
4,Sapiens: A Brief History ...,54.23,Five


Extract 3 pages:

In [364]:
stub = "http://books.toscrape.com/catalogue/page-{}.html"

In [366]:
#check the 2nd page
stub.format(2)

'http://books.toscrape.com/catalogue/page-2.html'

In [367]:
table = []
for page in tqdm(range(3)):
    url = stub.format(page)
    html = requests.get(url).text
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table.append(get_table(soup))

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.38it/s]


In [368]:
len(table)

3

In [369]:
#Step3: combine those and check the shape now:
data = pd.concat(table, axis=0)
data.shape

(40, 3)

Children books: Scrap multiple pages(3) of this section from the same website mentioned in the abive above code

In [371]:
stub="http://books.toscrape.com/catalogue/category/books/childrens_11/page-{}.html"

In [377]:
stub.format(3)

'http://books.toscrape.com/catalogue/category/books/childrens_11/page-3.html'

In [373]:
table = []
for page in tqdm(range(3)):
    url = stub.format(page)
    html = requests.get(url).text
    soup = bs4.BeautifulSoup(html, 'html.parser')
    table.append(get_table(soup))

100%|████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:02<00:00,  1.41it/s]


In [374]:
data = pd.concat(table, axis=0)
data.shape

(29, 3)

In [378]:
data.head()

,Books,Price,Rating
0,Birdsong: A Story in ...,54.64,Three
1,The Bear and the ...,36.89,One
2,The Secret of Dreadwillow ...,56.13,One
3,The White Cat and ...,58.08,Four
4,Little Red,13.47,Three
